In [1]:
%%capture --no-stderr
%pip install -U langchain_community langchain_openai langchain_fireworks langgraph wikipedia duckduckgo-search tavily-python

In [37]:
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_core.tools import tool

In [61]:
from langchain_community.tools import DuckDuckGoSearchResults


In [62]:
# DDG
# search_engine = DuckDuckGoSearchAPIWrapper()

search_engine = DuckDuckGoSearchResults()

In [39]:
results = DuckDuckGoSearchAPIWrapper()._ddgs_text("aa")

In [40]:
DuckDuckGoSearchAPIWrapper().run("the winner of the 2024 Men's Australian Open")


'Australian Open men\'s final 2024: Jannik Sinner beats Daniil Medvedev in Melbourne final. Published. 28 January 2024. ... Sinner clinched victory with a forehand winner down the line, falling to ... The world No.4 clinched the Australian Open 2024 men\'s singles title in stunning fashion, overcoming third seed Daniil Medvedev 3-6 3-6 6-4 6-4 6-3 in three hours and 44 minutes on Sunday night at Rod Laver Arena. MORE: All the stats from the match Jannik Sinner came back from two sets down to beat Daniil Medvedev 3-6, 3-6, 6-4, 6-4, 6-3 in the Australian Open men\'s singles final, earning him his first ever Grand Slam title.The 22-year-old ... Australian Open 2024 took place from 14-28 Jan 2024 Jannik Sinner beat Daniil Medvedev check the winners list of Men & Women Singles, Doubles, & Mixed Doubles. More from the 2024 Australian Open winner Jannik Sinner: "I want to thank all the sponsors, ball kids, chair umpire, everyone who makes this event so good. It\'s the happiest Slam, it is a 

In [41]:
results


[{'title': 'Alcoholics Anonymous - Wikipedia',
  'href': 'https://en.wikipedia.org/wiki/Alcoholics_Anonymous',
  'body': 'Alcoholics Anonymous (AA) is a global, peer-led mutual-aid fellowship focused on an abstinence-based recovery model from alcoholism through its spiritually-inclined twelve-step program. [1] The organization adheres to Twelve Traditions that emphasize anonymity, the absence of a hierarchical structure, and principles of being free to all, non-promotional, non-professional, unaffiliated, non ...'},
 {'title': 'American Airlines maintains AAdvantage status and reward levels for ...',
  'href': 'https://news.aa.com/news/news-details/2024/American-Airlines-maintains-AAdvantage-status-and-reward-levels-for-second-year-in-a-row-AADV-12/default.aspx',
  'body': "American Airlines continues to make the AAdvantage® program more rewarding for its members as it updates the industry's longest-running loyalty program for 2025. For the second year in a row, the requirements for st

# Setup

In [42]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()
# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")


# Define graph state

In [43]:
from typing import List
from typing_extensions import TypedDict


class ReWOO(TypedDict):
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str

# Planner

In [44]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
# model = ChatOpenAI(model="gpt-4o")
model = ChatAnthropic(model="claude-3-5-sonnet-20240620")


In [45]:
prompt = """For the following task, make plans that can solve the problem step by step. For each plan, indicate \
which external tool together with tool input to retrieve evidence. You can store the evidence into a \
variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
(1) Duckduckgo[input]: Worker that searches results from Duckduckgo. Useful when you need to find short
and succinct answers about a specific topic. The input should be a search query.
(2) LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general
world knowledge and common sense. Prioritize it when you are confident in solving the problem
yourself. Input can be any instruction.

For example,
Task: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked x
hours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hours
less than Toby. How many hours did Rebecca work?
Plan: Given Thomas worked x hours, translate the problem into algebraic expressions and solve
with Wolfram Alpha. #E1 = WolframAlpha[Solve x + (2x − 10) + ((2x − 10) − 8) = 157]
Plan: Find out the number of hours Thomas worked. #E2 = LLM[What is x, given #E1]
Plan: Calculate the number of hours Rebecca worked. #E3 = Calculator[(2 ∗ #E2 − 10) − 8]

Begin! 
Describe your plans with rich details. Each Plan should be followed by only one #E.

Task: {task}"""

In [46]:
task = "what is the exact hometown of the 2024 mens australian open winner"


In [47]:
result = model.invoke(prompt.format(task=task))


In [48]:
prompt.format(task=task)


'For the following task, make plans that can solve the problem step by step. For each plan, indicate which external tool together with tool input to retrieve evidence. You can store the evidence into a variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)\n\nTools can be one of the following:\n(1) Duckduckgo[input]: Worker that searches results from Duckduckgo. Useful when you need to find short\nand succinct answers about a specific topic. The input should be a search query.\n(2) LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general\nworld knowledge and common sense. Prioritize it when you are confident in solving the problem\nyourself. Input can be any instruction.\n\nFor example,\nTask: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked x\nhours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hours\nless than Toby. How many hours did Rebecca work?\nPlan: Given Thoma

In [49]:
print(result.content)


Here's a step-by-step plan to solve the task:

Plan: First, we need to find out who won the 2024 Men's Australian Open tennis tournament. This is recent information, so we'll use a search engine to get the most up-to-date result.
#E1 = Duckduckgo[Who won the 2024 Men's Australian Open tennis tournament?]

Plan: Now that we have the winner's name from #E1, we need to search for information about their hometown. We'll use a search query that specifically asks for the player's hometown or birthplace.
#E2 = Duckduckgo[What is the exact hometown or birthplace of {winner from #E1}?]

Plan: If the previous search doesn't provide a clear or specific enough answer about the exact hometown, we'll need to cross-reference the information. We'll search for the player's biography or profile on official tennis websites or reputable sports databases.
#E3 = Duckduckgo[{winner from #E1} tennis player biography birthplace]

Plan: To ensure accuracy, we'll compare the information from #E2 and #E3. If ther

### Planner Node

In [50]:
import re

from langchain_core.prompts import ChatPromptTemplate


In [51]:
# Regex to match expressions of the form E#... = ...[...]
regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"
prompt_template = ChatPromptTemplate.from_messages([("user", prompt)])
planner = prompt_template | model

In [52]:
def get_plan(state: ReWOO): # state by a TypedDict
    task = state["task"]
    result = planner.invoke({"task": task})
    # Find all matches in the sample text
    matches = re.findall(regex_pattern, result.content)
    return {"steps": matches, "plan_string": result.content}

# Executor

In [81]:
# DDG
# search = DuckDuckGoSearchAPIWrapper()

search = DuckDuckGoSearchResults()


In [82]:
def _get_current_task(state: ReWOO):
    if "results" not in state or state["results"] is None:
        return 1
    if len(state["results"]) == len(state["steps"]):
        return None
    else:
        return len(state["results"]) + 1

In [83]:
# DuckDuckGoSearchAPIWrapper()._ddgs_text("aa")


In [84]:
def tool_execution(state: ReWOO):
    """Worker node that executes the tools of a given plan."""
    _step = _get_current_task(state)
    _, step_name, tool, tool_input = state["steps"][_step - 1]
    _results = (state["results"] or {}) if "results" in state else {}
    
    # Replace any result placeholders in the tool_input
    for k, v in _results.items():
        tool_input = tool_input.replace(k, v)
    
    if tool == "Duckduckgo":
        # Clean up the query if it's too complex
        if isinstance(tool_input, str):
            # Remove any obvious JSON-like structure but keep the actual query
            if tool_input.startswith('{"') or tool_input.startswith('[{'):
                try:
                    import json
                    data = json.loads(tool_input)
                    if isinstance(data, dict):
                        tool_input = data.get('query', '')
                    elif isinstance(data, list) and len(data) > 0:
                        # If it's a list, assume the query is in a specific field
                        # Adjust this based on your actual data structure
                        tool_input = data[0].get('query', '')
                except json.JSONDecodeError:
                    # If JSON parsing fails, try to extract text between quotes
                    import re
                    matches = re.findall(r'"([^"]*)"', tool_input)
                    if matches:
                        tool_input = matches[0]
        
        result = search.invoke(tool_input)
    elif tool == "LLM":
        result = model.invoke(tool_input)
    else:
        raise ValueError
        
    _results[step_name] = str(result)
    return {"results": _results}

# Solver


In [85]:
solve_prompt = """Solve the following task or problem. To solve the problem, we have made step-by-step Plan and \
retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might \
contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer
directly with no extra words.

Task: {task}
Response:"""


def solve(state: ReWOO):
    plan = ""
    for _plan, step_name, tool, tool_input in state["steps"]:
        _results = (state["results"] or {}) if "results" in state else {}
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
            step_name = step_name.replace(k, v)
        plan += f"Plan: {_plan}\n{step_name} = {tool}[{tool_input}]"
    prompt = solve_prompt.format(plan=plan, task=state["task"])
    result = model.invoke(prompt)
    return {"result": result.content}

# Define Graph

In [86]:
def _route(state):
    _step = _get_current_task(state)
    if _step is None:
        # We have executed all tasks
        return "solve"
    else:
        # We are still executing tasks, loop back to the "tool" node
        return "tool"

In [87]:
from langgraph.graph import END, StateGraph, START

graph = StateGraph(ReWOO)
graph.add_node("plan", get_plan)
graph.add_node("tool", tool_execution)
graph.add_node("solve", solve)
graph.add_edge("plan", "tool")
graph.add_edge("solve", END)
graph.add_conditional_edges("tool", _route)
graph.add_edge(START, "plan")

app = graph.compile()

In [88]:
for s in app.stream({"task": task}):
    print(s)
    print("---")
    

{'plan': {'steps': [("First, we need to identify the winner of the 2024 Men's Australian Open tennis tournament. This is a recent event, so we'll use a search engine to find this information. ", '#E1', 'Duckduckgo', "Who won the 2024 Men's Australian Open tennis tournament?"), ("Now that we have the winner's name from #E1, we need to find out their exact hometown. Let's search for this specific information.", '#E2', 'Duckduckgo', 'Exact hometown of #E1'), ("If the previous search doesn't provide a clear answer about the exact hometown, we might need to cross-reference the information or look for more detailed sources. Let's try a more specific search query.", '#E3', 'Duckduckgo', '#E1 tennis player birthplace'), ("If we still don't have a definitive answer, we can use the LLM to analyze the information we've gathered so far and provide a best estimate or highlight any inconsistencies in the data.", '#E4', 'LLM', "Based on the information from #E1, #E2, and #E3, what is the most likely 

DuckDuckGoSearchException: https://lite.duckduckgo.com/lite/ return None. params=None content=None data={'q': "Exact hometown of snippet: Jannik Sinner came back from two sets down to beat Daniil Medvedev 3-6, 3-6, 6-4, 6-4, 6-3 in the Australian Open men's singles final, earning him his first ever Grand Slam title.The 22-year-old ..., title: Australian Open 2024: Jannik Sinner, Aryna Sabalenka crowned as Grand ..., link: https://www.cbssports.com/tennis/news/australian-open-2024-jannik-sinner-aryna-sabalenka-crowned-as-grand-slam-singles-champions-at-melbourne-park/, snippet: Jannik Sinner is a Grand Slam champion.. The world No.4 clinched the Australian Open 2024 men's singles title in stunning fashion, overcoming third seed Daniil Medvedev 3-6 3-6 6-4 6-4 6-3 in three hours and 44 minutes on Sunday night at Rod Laver Arena.. MORE: All the stats from the match, title: Sinner, winner: Italian takes first major at AO 2024 | AO - Australian Open, link: https://ausopen.com/articles/news/

In [36]:
# Print out the final result
print(s["solve"]["result"])

KeyError: 'solve'